# How do I mount files from my own _cloud storage_?
### Overview
Connecting to cloud storage facilitates sharing and leveraging bioinformatics data that is commonly stored there (details [here](http://docs.sevenbridges.com/docs/connecting-cloud-storage-overview)). Here, we show an example of **reading** files from the _Seven Bridges Platform_ from a users _Amazon Web Services_ **bucket**. Afterwards, an **alias** to the file will be created in the user's _Seven Bridges Platform_ project.

### Prerequisites
 1. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 2. You need to have an AWS account with an S3 bucket
 3. You have some files on this bucket, e.g. by following this [recipe](volumes_writeToCloudStorage.ipynb)
 3. You need to be a member (or owner) of a project, e.g. by following this [recipe](projects_makeNew.ipynb)
 
### Setting up Identity and Access Management (IAM) access 
There is a detailed tutorial to set up access to your buckets on [AWS](http://docs.sevenbridges.com/v1.0/docs/aws-cloud-storage-tutorial) or [GCP](https://docs.sevenbridges.com/docs/google-cloud-storage-gcs-volumes). 

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the credentials file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify credentials file profile {cgc, sbg, default}
prof = 'default'

config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Mount the volume
Since we have configured the IAM policy in the prior step, we can now mount the volume. Make sure to **replace** (BUCKET-NAME, INSERT ACCESS KEY ID, and INSERT SECRET ACCESS KEY) below

### Warning
You only need to mount a volume **once**. If you have already done this, then skip this cell and go to _Get details of the volume_. If you try to mount it more than once, you will get an error (likely error: `Forbidden: Could not obtain cross-write access on the service. Check the documentation.`).

#### Additional information
 * Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/create-a-volume-v2)

In [ ]:
volume_import = api.volumes.create_s3_volume(
    name='my_volume_RO', 
    bucket='BUCKET-NAME',
    access_key_id='INSERT ACCESS KEY ID',
    secret_access_key = 'INSERT SECRET ACCESS KEY',                             
    access_mode='RO')

## Get details of the volume
Let's list all our volumes and get one volume's (`my_volume_RO`, which we have just imported in the prior cell) details to make sure it's mounted as we expected.

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here (list)](http://docs.sevenbridges.com/docs/list-volumes-v2) and [here (details)](http://docs.sevenbridges.com/docs/get-details-of-a-volume-v2)

In [ ]:
# [USER INPUT] Set volume name here:
volume_name = 'my_volume_RO'  


all_volumes = api.volumes.query()

my_volume = [v for v in all_volumes.all() 
            if v.name == volume_name]

if not my_volume:
    print('Volume {} does not exist, check name / mounting'
          .format(volume_name))
    raise KeyboardInterrupt
else:
    my_volume = my_volume[0]

print(my_volume.name)
print(dir(my_volume))

## Set up a new project
We will first make an empty project and verify that there are no files in it.


#### PROTIPS
 * A detailed _recipe_ for creating projects is [here](projects_makeNew.ipynb)
 * A detailed _recipe_ for listing files is [here](files_listAll.ipynb)

In [ ]:
# [USER INPUT] Set project name here:
new_project_name = '_MCA'  


# What are my funding sources?
billing_groups = api.billing_groups.query()  

# check if this project already exists. LIST all projects and check for name match
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == new_project_name]      
              
if my_project:    
    print('A project with the name {} already exists, skipping creation'
          .format(new_project_name))
    my_project = my_project[0]
else:
    # CREATE the new project
    new_project = {
            'billing_group': billing_groups[-1].id,
            'name': new_project_name,
            'description': """
            A project created by the API recipe (volumes_readFromCloudStorage.ipynb).
            This also supports **markdown**
            _Pretty cool_, right?
            """
    }
    
    my_project = api.projects.create(name=new_project['name'], 
                                     billing_group=new_project['billing_group'], 
                                     description=new_project['description'], 
                                     )
    
    # (re)list all projects, and get your new project
    my_project = [p for p in api.projects.query(limit=100).all() 
              if p.name == new_project_name][0]

    print('Your new project {} has been created.'.format(my_project.name))
    if hasattr(my_project, 'description'):
        print('Project description: {} \n'.format(my_project.description)) 
        
my_files = api.files.query(limit=100, project=my_project)
if my_files.total == 0:
    print('no files in the project')

## Start an import job
Spin up a job to import files to the _Seven Bridges Platform_ from your _Cloud Storage_ (Volume).

#### PROTIPS
 * If you followed this [recipe](volumes_writeToCloudStorage.ipynb) to export files to your S3 bucket, then **you don't need to change** names in the _file\_list_ below.
 * If not, this file_list **must be adjusted** to match your bucket.

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](https://docs.sevenbridges.com/reference#start-a-bulk-import-job)

In [ ]:
# [USER INPUT] Set S3 file names here:
file_list = ['files_listAll.ipynb',
            'files_copyFromMyProject.ipynb',
            'files_copyFromPublicReference.ipynb']


# Loop through selected files, start one job for each.
imports=[]
for f_name in file_list:
    # Submit import jobs
    imports.append({
            'volume': volume,
            'location': '/data/{}'.format(f_name),
            'project': project,
            'name': f_name,
            'overwrite': False
        })
import_jobs = api.imports.bulk_submit(imports=imports)


## List and detail import jobs
The `get_imports` method will capture all import which come into my_project from my_volume

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](http://docs.sevenbridges.com/docs/list-import-jobs-v2) and [here](https://docs.sevenbridges.com/reference#get-details-of-a-bulk-import-job)

In [ ]:
import_jobs = api.imports.bulk_get(imports=imports)
for j in import_jobs:
    print('File {} import stated {}; it is {}'
         .format(j.destination, j.started_on, j.state))
print('\n')